In [1]:
import csv
from graph_tool import Graph
import pandas as pd
import netpixi
from netpixi.data.gt import *
import seaborn as sns
from graph_tool import centrality
from graph_tool import clustering
import matplotlib.pyplot as plt
sns.set()

In [2]:
donations = pd.read_csv('vacinas_certo.csv')
donations_drop = donations[['Donating Entity','Donated to','Donation Amount']]
donations_drop.head()

,Donating Entity,Donated to,Donation Amount
0,Alrosa Group,Angola,"50,000"
1,Alrosa Group,Zimbabwe,"50,000"
2,Association of Healthcare Providers,India,"100,000,000*"
3,Australia,Unknown,"38,902,000"
4,Australia,Timor-Leste,"30,000"


In [3]:
pib = pd.read_csv('pib_certo.csv')
pib.head()

,Donating Entity,2020
0,Aruba,NaN
1,Africa Eastern and Southern,9.008290e+11
2,Afghanistan,1.980707e+10
3,Africa Western and Central,7.865850e+11
4,Angola,6.230691e+10


In [4]:
df_final = pd.merge(donations_drop, pib, 
                   on='Donating Entity', 
                   how='left')
df_final.head()

,Donating Entity,Donated to,Donation Amount,2020
0,Alrosa Group,Angola,"50,000",NaN
1,Alrosa Group,Zimbabwe,"50,000",NaN
2,Association of Healthcare Providers,India,"100,000,000*",NaN
3,Australia,Unknown,"38,902,000",1.330900e+12
4,Australia,Timor-Leste,"30,000",1.330900e+12


In [5]:
df_novo = pd.merge(df_final, pib, 
                   how='left', left_on = 'Donated to', right_on = 'Donating Entity')

df_novo.drop('Donating Entity_y', axis='columns', inplace=True)
df_finalfinal = df_novo.dropna(0,'any') 
df_finalfinal

,Donating Entity_x,Donated to,Donation Amount,2020_x,2020_y
4,Australia,Timor-Leste,"30,000",1.330900e+12,1.821000e+09
7,Australia,Fiji,"1,000,000",1.330900e+12,4.376015e+09
8,Australia,Solomon Islands,"60,000",1.330900e+12,1.551246e+09
9,Australia,Papua New Guinea,"8,000",1.330900e+12,2.359152e+10
13,Azerbaijan,Tajikistan,"40,000",4.260718e+10,8.194150e+09
...,...,...,...,...,...
373,United States,Ukraine,"428,571",2.093660e+13,1.555820e+11
375,United States,West Bank and Gaza,"428,571",2.093660e+13,1.556130e+10
380,United States,Canada,"1,500,000",2.093660e+13,1.644040e+12
381,United States,Mexico,"2,500,000",2.093660e+13,1.076160e+12


In [6]:
df = df_finalfinal.rename(columns = {'Donating Entity_x': 'Donating Entity', '2020_x': 'PIB_doador', '2020_y': 'PIB_doador'}, 
                     inplace = False)

In [7]:
df['amount'] = df['Donation Amount'].str.replace(',', '').astype(float)
df.drop('Donation Amount', axis='columns', inplace=True)
df = df.rename(columns = {'amount': 'Donation Amount'}, inplace = False)
df

,Donating Entity,Donated to,PIB_doador,PIB_doador,Donation Amount
4,Australia,Timor-Leste,1.330900e+12,1.821000e+09,30000.0
7,Australia,Fiji,1.330900e+12,4.376015e+09,1000000.0
8,Australia,Solomon Islands,1.330900e+12,1.551246e+09,60000.0
9,Australia,Papua New Guinea,1.330900e+12,2.359152e+10,8000.0
13,Azerbaijan,Tajikistan,4.260718e+10,8.194150e+09,40000.0
...,...,...,...,...,...
373,United States,Ukraine,2.093660e+13,1.555820e+11,428571.0
375,United States,West Bank and Gaza,2.093660e+13,1.556130e+10,428571.0
380,United States,Canada,2.093660e+13,1.644040e+12,1500000.0
381,United States,Mexico,2.093660e+13,1.076160e+12,2500000.0


In [8]:
g = Graph(directed=True)

In [9]:
# Criando uma propriedade para os vértices de Novo Jogador
g.vp['Nome País'] = g.new_vp('object')

lista_paises = []

for i in df['Donating Entity']:
    if i not in lista_paises:
        lista_paises.append(i)
        
for i in df['Donated to']:
    if i not in lista_paises:
        lista_paises.append(i)
    
for i in range(len(lista_paises)):
    g.add_vertex()
    g.vp['Nome País'][i] = lista_paises[i]

In [10]:
for e in g.vertices():
    print(g.vp['Nome País'][e])

print(len(lista_paises))

Australia
Azerbaijan
Barbados
Chile
China
France
Germany
India
Ireland
Israel
Japan
Lithuania
Mexico
Morocco
New Zealand
Poland
Portugal
Romania
Serbia
South Africa
Turkey
United States
Timor-Leste
Fiji
Solomon Islands
Papua New Guinea
Tajikistan
Uzbekistan
Guyana
Ecuador
Malaysia
Thailand
Vietnam
Cambodia
Kiribati
Philippines
Myanmar
Sri Lanka
Pakistan
Nepal
El Salvador
Namibia
North Macedonia
Afghanistan
Chad
Congo, Rep.
Djibouti
Bolivia
Equatorial Guinea
Georgia
Guinea
Iraq
Sierra Leone
Gabon
Togo
Algeria
Angola
Belarus
Dominican Republic
Ethiopia
Lebanon
Maldives
Mauritania
Mongolia
Montenegro
Niger
Sudan
Tunisia
Zimbabwe
Bangladesh
Benin
Botswana
Cameroon
Mauritius
Somalia
Ukraine
Moldova
Cote d'Ivoire
Bhutan
Jamaica
Paraguay
Albania
Belize
Dominica
Eswatini
Ghana
Guatemala
Kenya
Malawi
Mozambique
Nicaragua
Nigeria
Rwanda
Senegal
Seychelles
St. Lucia
Suriname
Uganda
Honduras
West Bank and Gaza
Indonesia
Cyprus
Libya
Lesotho
Liberia
Madagascar
Panama
Peru
Colombia
Costa Rica
Argent

In [11]:
from graph_tool import util

g.ep['Número de doações'] = g.new_ep('object')

for pais_doador, pais_receptor, index in zip(df['Donating Entity'], df['Donated to'], df.index):
    str_index_doador = util.find_vertex(g,g.vp['Nome País'],pais_doador) 
    index_doador = int(str(str_index_doador).split(" ")[4].replace("'","")) #pega o índice do vencedor
    
    str_index_receptor = util.find_vertex(g,g.vp['Nome País'],pais_receptor) 
    index_receptor = int(str(str_index_receptor).split(" ")[4].replace("'","")) #pega o índice do vencedor
    
    aresta_existe = 0
    for e in g.edges():
        # Se não tem valor, inicializa com 1 vitória
        if g.ep['Número de doações'][e] == None:
            g.ep['Número de doações'][e] = 0

        # Se a aresta já existir, incrementa o número de vitórias
        if e.source() == index_doador and e.target() == index_receptor:
            g.ep['Número de doações'][e] += int(df['Donation Amount'][index])
            aresta_existe = 1

    # Se a aresta não existe, cria uma nova
    if aresta_existe == 0:
        g.add_edge(index_doador, index_receptor)
        
        e = g.edge(index_doador, index_receptor)
        g.ep['Número de doações'][e] = df['Donation Amount'][index]
        print(df['Donation Amount'][index])

30000.0
1000000.0
60000.0
8000.0
40000.0
50000.0
3000.0
20000.0
1000000.0
1500000.0
3000000.0
3000000.0
100000.0
90000.0
1000000.0
2500000.0
500000.0
500000.0
1600000.0
300000.0
1500000.0
100000.0
800000.0
700000.0
200000.0
100000.0
300000.0
100000.0
100000.0
100000.0
200000.0
20000.0
250000.0
200000.0
100000.0
200000.0
200000.0
200000.0
150000.0
30000.0
100000.0
50000.0
300000.0
50000.0
200000.0
50000.0
300000.0
30000.0
400000.0
250000.0
100000.0
400000.0
600000.0
100000.0
200000.0
200000.0
20000.0
100000.0
200000.0
218400.0
25000.0
25000.0
25000.0
25000.0
100000.0
50000.0
100000.0
400000.0
100000.0
100000.0
100000.0
500000.0
50000.0
100000.0
25000.0
30000.0
70000.0
30000.0
20000.0
100000.0
50000.0
200000.0
80000.0
100000.0
50000.0
100000.0
150000.0
100000.0
200000.0
30000.0
200000.0
25000.0
100000.0
50000.0
25000.0
50000.0
500000.0
25000.0
50000.0
100000.0
35000.0
335500.0
5000.0
5000.0
3000000.0
1000000.0
1000000.0
1000000.0
1000000.0
1050000.0
15000.0
11000.0
100000.0
100800.0
1500

In [12]:
for e in g.edges():
    print(g.ep['Número de doações'][e])


30000.0
1000000.0
60000.0
8000.0
40000.0
50000.0
3000.0
20000.0
1500000.0
2500000.0
3500000.0
6903000.0
100000.0
90000.0
2000000.0
3000000.0
2700000.0
5500000.0
3400000.0
300000.0
1650000.0
100000.0
1400000.0
1100000.0
200000.0
100000.0
300000.0
300000.0
100000.0
200000.0
400000.0
40000.0
350000.0
200000.0
100000.0
200000.0
200000.0
200000.0
150000.0
30000.0
100000.0
50000.0
300000.0
50000.0
418000.0
50000.0
300000.0
30000.0
400000.0
250000.0
100000.0
800000.0
600000.0
100000.0
200000.0
200000.0
20000.0
100000.0
200000.0
218400.0
25000.0
25000.0
25000.0
25000.0
200000.0
50000.0
3300000.0
550000.0
200000.0
1100000.0
200000.0
500000.0
50000.0
100000.0
25000.0
30000.0
70000.0
30000.0
20000.0
100000.0
50000.0
200000.0
80000.0
100000.0
50000.0
100000.0
150000.0
100000.0
1700000.0
30000.0
200000.0
25000.0
100000.0
50000.0
25000.0
50000.0
500000.0
25000.0
50000.0
100000.0
35000.0
335500.0
5000.0
5000.0
3000000.0
1000000.0
1000000.0
1000000.0
1000000.0
1050000.0
15000.0
11000.0
100000.0
100800

In [15]:
# g = load_gt('../datasets/gt-polbooks.net.gz')
# m = draw_gt.sfdp_layout(g)
# move_gt(g, m)
save_gt(g, 'doacao_vacinas.net.gz')
r = netpixi.render('doacao_vacinas.net.gz');

In [16]:
r.vertex_set_key('Nome País')